In [3]:
# =============================================================================
# ADAPTIVE LSTM V2: CLASS-BASED SAFETY LAYER
# Architecture: Deep Learning Median -> Smart Floor -> Adaptive Risk Buffer
# =============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import warnings
import os
import random

warnings.filterwarnings('ignore')
os.environ['PYTHONHASHSEED'] = '42'
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# === CONFIGURATION ===
FILE_PATH = "CaRDS.csv"
TEST_SIZE = 0.2
VAL_SIZE = 0.15
SEQUENCE_LENGTH = 14
BATCH_SIZE = 64
EPOCHS = 40
QUANTILE = 0.5 # Median Prediction

# CẤU HÌNH LỚP AN TOÀN
SAFETY_CONFIG = {
    'summer_months': [6, 7, 8],
    'floor': {
        'enabled': True,
        'yoy_growth_min': 1.02,
        'mom_drop_max': 0.92,
        'mom_drop_summer': 0.95,
        'mom_drop_fall': 0.75
    },
    'buffer': {
        'enabled': True,
        'base_sigma': 2.0,
        'hist_coverage': 1.05,
        'summer_coverage': 1.15,
        'max_cap_pct': 0.50
    }
}

print("="*70)
print("ADAPTIVE LSTM PIPELINE WITH SAFETY LAYER V2")
print("="*70)

# =============================================================================
# 0. SAFETY LAYER CLASS
# =============================================================================
class WaterDemandSafetyLayer:
    def __init__(self, config):
        self.config = config
        self.risk_map = {} 

    def fit(self, df_val, y_true, y_pred_raw):
        print("   ⚙️ Fitting Safety Layer (Learning Risk Profiles)...")
        analysis = df_val[['PWSID_enc', 'Month']].copy()
        analysis['Actual'] = y_true.values if hasattr(y_true, 'values') else y_true
        analysis['Pred'] = y_pred_raw
        analysis['Shortage'] = analysis['Actual'] - analysis['Pred']
        
        risk_profile = analysis.groupby('PWSID_enc')['Shortage'].std().reset_index()
        risk_profile.rename(columns={'Shortage': 'Error_Std'}, inplace=True)
        
        shortage_only = analysis[analysis['Shortage'] > 0]
        if not shortage_only.empty:
            max_risk = shortage_only.groupby('PWSID_enc')['Shortage'].max().reset_index()
            max_risk.rename(columns={'Shortage': 'Max_Shortage'}, inplace=True)
        else:
            max_risk = pd.DataFrame(columns=['PWSID_enc', 'Max_Shortage'])
            
        summer_months = self.config['summer_months']
        summer_data = analysis[analysis['Month'].isin(summer_months)]
        summer_shortage = summer_data[summer_data['Shortage'] > 0]
        if not summer_shortage.empty:
            max_summer_risk = summer_shortage.groupby('PWSID_enc')['Shortage'].max().reset_index()
            max_summer_risk.rename(columns={'Shortage': 'Max_Summer_Shortage'}, inplace=True)
        else:
            max_summer_risk = pd.DataFrame(columns=['PWSID_enc', 'Max_Summer_Shortage'])

        profile = risk_profile.merge(max_risk, on='PWSID_enc', how='left')
        profile = profile.merge(max_summer_risk, on='PWSID_enc', how='left')
        profile = profile.fillna(0)
        self.risk_map = profile.set_index('PWSID_enc').to_dict('index')
        print(f"      -> Learned profiles for {len(self.risk_map)} stations.")

    def _apply_guardrails(self, base_pred, df_context):
        if not self.config['floor']['enabled']:
            return base_pred, np.zeros_like(base_pred, dtype=bool)

        cfg = self.config['floor']
        lag_12 = df_context['lag_12'].values
        lag_1 = df_context['lag_1'].values
        months = df_context['Month'].values
        
        floor_yoy = lag_12 * cfg['yoy_growth_min']
        floor_mom = lag_1 * cfg['mom_drop_max'] 
        
        summer_mask = np.isin(months, self.config['summer_months'])
        floor_mom[summer_mask] = lag_1[summer_mask] * cfg['mom_drop_summer']
        
        fall_mask = np.isin(months, [9, 10, 11])
        floor_mom[fall_mask] = lag_1[fall_mask] * cfg['mom_drop_fall']
        
        floored_pred = np.maximum.reduce([base_pred, floor_yoy, floor_mom])
        is_floored = floored_pred > (base_pred + 1e-3)
        return floored_pred, is_floored

    def _apply_risk_buffer(self, pred, df_context):
        if not self.config['buffer']['enabled']:
            return pred, np.zeros_like(pred)

        cfg = self.config['buffer']
        pwsid_encs = df_context['PWSID_enc'].values
        months = df_context['Month'].values
        
        buffers = []
        for i, pid in enumerate(pwsid_encs):
            profile = self.risk_map.get(pid, {'Error_Std': 0, 'Max_Shortage': 0, 'Max_Summer_Shortage': 0})
            
            base_buf = cfg['base_sigma'] * profile['Error_Std']
            hist_buf = profile['Max_Shortage'] * cfg['hist_coverage']
            
            summer_buf = 0
            if months[i] in self.config['summer_months']:
                summer_buf = profile['Max_Summer_Shortage'] * cfg['summer_coverage']
            
            final_buf = max(base_buf, hist_buf, summer_buf)
            
            if pred[i] > 0:
                cap_val = pred[i] * cfg['max_cap_pct']
                if final_buf > cap_val and final_buf > profile['Max_Shortage'] * 1.1:
                    final_buf = cap_val
            
            buffers.append(final_buf)
            
        return pred + np.array(buffers), np.array(buffers)

    def predict(self, raw_pred, df_context, explain=False):
        floored_pred, is_floored = self._apply_guardrails(raw_pred, df_context)
        final_pred, buffer_amount = self._apply_risk_buffer(floored_pred, df_context)
        
        if explain:
            explanation_df = df_context.copy()
            explanation_df['Raw_Model'] = raw_pred
            explanation_df['Floored_Pred'] = floored_pred
            explanation_df['Is_Floored'] = is_floored
            explanation_df['Risk_Buffer'] = buffer_amount
            explanation_df['Final_Forecast'] = final_pred
            return final_pred, explanation_df
        return final_pred

# =============================================================================
# 1. DATA & FEATURE ENGINEERING
# =============================================================================
def clean_physics_based(series):
    median_val = series.median()
    if pd.isna(median_val) or median_val <= 0: return series.fillna(0)
    phys_min = median_val * 0.05
    phys_max = median_val * 10.0
    mask_invalid = (series < phys_min) | (series > phys_max)
    if mask_invalid.any():
        series_clean = series.copy()
        series_clean[mask_invalid] = np.nan
        return series_clean.interpolate(method='linear', limit_direction='both')
    return series

def load_and_process_data(file_path):
    print(f"\n📂 Loading data from {file_path}...")
    if not os.path.exists(file_path):
        print(f"❌ Error: File not found.")
        return None
    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"❌ Error reading file: {e}")
        return None

    df['Variable'] = df['Variable'].astype(str).str.strip().str.lower()
    date_cols = [c for c in df.columns if c not in ['PWSID', 'Variable']]
    df_melt = df.melt(id_vars=['PWSID', 'Variable'], value_vars=date_cols, var_name='Date', value_name='Value')
    df_pivot = df_melt.pivot_table(index=['PWSID', 'Date'], columns='Variable', values='Value').reset_index()
    
    rename_map = {'demand': 'Demand', 'temperature': 'Temperature', 'precipitation': 'Precipitation', 'pdsi': 'PDSI'}
    df_pivot.rename(columns=rename_map, inplace=True)
    df_pivot['Date'] = pd.to_datetime(df_pivot['Date'])
    df_final = df_pivot.sort_values(['PWSID', 'Date']).reset_index(drop=True)
    
    for col in ['Temperature', 'Precipitation', 'PDSI']:
        if col in df_final.columns:
            val = 0 if col == 'Precipitation' else df_final[col].median()
            df_final[col] = df_final[col].fillna(val)
    
    if 'Demand' in df_final.columns:
        df_final['Demand'] = df_final.groupby('PWSID')['Demand'].transform(clean_physics_based)
        df_final['Demand'] = df_final['Demand'].fillna(0)
    return df_final

df = load_and_process_data(FILE_PATH)
if df is None: exit()

# Feature Engineering
print("\n🔨 Creating features for LSTM...")
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
df['Is_Summer_Peak'] = ((df['Month'] >= 6) & (df['Month'] <= 8)).astype(int)

le = LabelEncoder()
df['PWSID_enc'] = le.fit_transform(df['PWSID'])

# Scaling (Only continuous vars)
scale_cols = ['Demand', 'Temperature', 'Precipitation', 'month_sin', 'month_cos']
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[scale_cols] = scaler.fit_transform(df[scale_cols])

feature_cols = scale_cols + ['PWSID_enc']
demand_mean = scaler.mean_[0]
demand_std = scaler.scale_[0]

# Sequence Creation
def create_sequences(df_scaled, df_raw, seq_len, feature_cols):
    X, y = [], []
    meta_info = [] 
    
    unique_ids = df_scaled['PWSID_enc'].unique()
    for pid in unique_ids:
        group = df_scaled[df_scaled['PWSID_enc'] == pid]
        group_raw = df_raw[df_raw['PWSID_enc'] == pid] 
        
        vals = group[feature_cols].values
        target = group['Demand'].values 
        dates = group_raw['Date'].values
        months = group_raw['Month'].values
        
        if len(group) <= seq_len: continue
            
        for i in range(len(group) - seq_len):
            X.append(vals[i : i+seq_len])
            y.append(target[i+seq_len])
            meta_info.append({
                'PWSID_enc': pid,
                'Date': dates[i+seq_len],
                'Month': months[i+seq_len],
                'Actual_Raw': df_raw[(df_raw['PWSID_enc']==pid) & (df_raw['Date']==dates[i+seq_len])]['Demand'].values[0]
            })
            
    return np.array(X), np.array(y), pd.DataFrame(meta_info)

print("   Processing sequences...")
X_all, y_all, meta_df = create_sequences(df_scaled, df, SEQUENCE_LENGTH, feature_cols)

# Split
unique_dates = meta_df['Date'].sort_values().unique()
n_test = int(len(unique_dates) * TEST_SIZE)
n_val = int(len(unique_dates) * VAL_SIZE)
test_start = unique_dates[-n_test]
val_start = unique_dates[-(n_test + n_val)]

train_mask = meta_df['Date'] < val_start
val_mask = (meta_df['Date'] >= val_start) & (meta_df['Date'] < test_start)
test_mask = meta_df['Date'] >= test_start

X_train, y_train = X_all[train_mask], y_all[train_mask]
X_val, y_val = X_all[val_mask], y_all[val_mask]
X_test, y_test = X_all[test_mask], y_all[test_mask]

meta_train = meta_df[train_mask].reset_index(drop=True)
meta_val = meta_df[val_mask].reset_index(drop=True)
meta_test = meta_df[test_mask].reset_index(drop=True)

print(f"   Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")

# =============================================================================
# 3. TRAINING LSTM
# =============================================================================
print("\n" + "="*70)
print("STEP 1: TRAINING LSTM (Quantile Loss Alpha=0.5)")
print("="*70)

def quantile_loss(q, y_true, y_pred):
    e = y_true - y_pred
    return tf.reduce_mean(tf.maximum(q*e, (q-1)*e))

def loss_func(y_true, y_pred):
    return quantile_loss(QUANTILE, y_true, y_pred)

model = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(64, return_sequences=True, activation='tanh'),
    Dropout(0.2),
    LSTM(32, activation='tanh'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=loss_func)

early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4)

print("   ⏳ Training LSTM Network...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# Predict & Inverse
pred_scaled_train = model.predict(X_train)
pred_scaled_val = model.predict(X_val)
pred_scaled_test = model.predict(X_test)

def inverse(y_scaled):
    return (y_scaled.flatten() * demand_std) + demand_mean

raw_pred_train = inverse(pred_scaled_train)
raw_pred_val = inverse(pred_scaled_val)
raw_pred_test = inverse(pred_scaled_test)

# =============================================================================
# 4. APPLYING SAFETY LAYER
# =============================================================================
print("\n" + "="*70)
print("STEP 2 & 3: APPLYING WATER DEMAND SAFETY LAYER")
print("="*70)

# IMPORTANT: Re-create lags for the meta DataFrame because sequence splitting broke continuity
def recreate_lags(df):
    df = df.copy()
    df = df.sort_values(['PWSID_enc', 'Date'])
    df['lag_1'] = df.groupby('PWSID_enc')['Actual_Raw'].shift(1)
    df['lag_12'] = df.groupby('PWSID_enc')['Actual_Raw'].shift(12)
    df = df.fillna(method='bfill')
    return df

meta_train = recreate_lags(meta_train)
meta_val = recreate_lags(meta_val)
meta_test = recreate_lags(meta_test)

# Initialize Safety Layer
safety_layer = WaterDemandSafetyLayer(config=SAFETY_CONFIG)

# 1. Fit (Using Actual_Raw from meta_df)
safety_layer.fit(meta_val, meta_val['Actual_Raw'], raw_pred_val)

# 2. Predict & Explain
print("   🚀 Generating forecasts with safety margins...")
final_val_pred, val_explain = safety_layer.predict(raw_pred_val, meta_val, explain=True)
final_test_pred, test_explain = safety_layer.predict(raw_pred_test, meta_test, explain=True)

print("\n🔍 AUDIT LOG SAMPLE (Test Set):")
print(test_explain.head(3)[['Raw_Model', 'Floored_Pred', 'Risk_Buffer', 'Final_Forecast']])

# =============================================================================
# 5. EVALUATION & VISUALIZATION (LSTM SPECIALIZED)
# =============================================================================
print("\n" + "="*70)
print("FINAL EVALUATION & VISUALIZATION")
print("="*70)

# 1. Mean Map
profile_df = pd.concat([meta_train, meta_val])
mean_map = profile_df.groupby('PWSID_enc')['Actual_Raw'].mean().to_dict()

# 2. Evaluation
def evaluate_model(y_true, y_pred, df_meta, dataset_name):
    if hasattr(y_true, 'values'): y_true = y_true.values
        
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    threshold = pd.Series(y_true).quantile(0.1)
    mask = y_true > threshold
    if mask.sum() > 0:
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / (y_true[mask] + 1e-10))) * 100
    else:
        mape = 0.0
    
    under_pred_count = np.sum(y_pred < y_true)
    under_rate = (under_pred_count / len(y_true)) * 100
    over_rate = 100 - under_rate
    avg_shortage = np.mean(np.maximum(y_true - y_pred, 0))
    avg_surplus = np.mean(np.maximum(y_pred - y_true, 0))
    
    months = df_meta['Month'].values
    summer_mask = (months >= 6) & (months <= 8)
    if np.sum(summer_mask) > 0:
        summer_under = np.sum((y_pred[summer_mask] < y_true[summer_mask])) / np.sum(summer_mask) * 100
    else:
        summer_under = 0.0
    
    print(f"\n📊 {dataset_name}:")
    print(f"   MAE: {mae:,.2f} | R²: {r2:.4f}")
    print(f"   MAPE (filtered): {mape:.2f}%")
    print(f"   ⚠️  Under-pred: {under_rate:.1f}% | Avg Shortage: {avg_shortage:,.2f}")
    print(f"   ✅ Over-pred: {over_rate:.1f}% | Avg Surplus: {avg_surplus:,.2f}")
    print(f"   🌞 Summer Under-pred: {summer_under:.1f}% (CRITICAL METRIC)")
    return {'R2': r2}

val_metrics = evaluate_model(meta_val['Actual_Raw'], final_pred_val, meta_val, "VALIDATION SET")
test_metrics = evaluate_model(meta_test['Actual_Raw'], final_pred_test, meta_test, "TEST SET")

# 3. Visualization
print("\n" + "="*60)
print("VISUALIZING RESULTS")
print("="*60)

val_plot = val_explain.copy()
val_plot['Actual'] = val_plot['Actual_Raw']
val_monthly = val_plot.groupby('Date')[['Actual', 'Final_Forecast']].sum().reset_index()

test_plot = test_explain.copy()
test_plot['Actual'] = test_plot['Actual_Raw']
test_monthly = test_plot.groupby('Date')[['Actual', 'Final_Forecast']].sum().reset_index()

fig, axes = plt.subplots(2, 1, figsize=(15, 10))

axes[0].plot(val_monthly['Date'], val_monthly['Actual'], 'k-o', label='Actual', alpha=0.7)
axes[0].plot(val_monthly['Date'], val_monthly['Final_Forecast'], 'g--s', label='Predicted (Adaptive LSTM)', alpha=0.8)
axes[0].fill_between(val_monthly['Date'], val_monthly['Final_Forecast'], val_monthly['Actual'],
                     where=(val_monthly['Final_Forecast'] < val_monthly['Actual']),
                     color='red', alpha=0.3, label='Shortage Risk')
axes[0].set_title(f"Validation Set (R2={val_metrics['R2']:.3f})")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(test_monthly['Date'], test_monthly['Actual'], 'k-o', label='Actual', alpha=0.7)
axes[1].plot(test_monthly['Date'], test_monthly['Final_Forecast'], 'b--s', label='Predicted (Adaptive LSTM)', alpha=0.8)
axes[1].fill_between(test_monthly['Date'], test_monthly['Final_Forecast'], test_monthly['Actual'],
                     where=(test_monthly['Final_Forecast'] < test_monthly['Actual']),
                     color='red', alpha=0.3, label='Shortage Risk')
axes[1].set_title(f"Test Set (R2={test_metrics['R2']:.3f})")
axes[1].legend()
axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Insight Plot
print("\nGenerating Margin Efficiency Plot...")
test_explain['Demand'] = test_explain['Actual_Raw']
test_explain['Margin_Pct'] = ((test_explain['Final_Forecast'] - test_explain['Demand']) / (test_explain['Demand']+1)) * 100
test_explain['Mean_Volume'] = test_explain['PWSID_enc'].map(mean_map)
test_explain['Is_Summer_Peak'] = ((test_explain['Month'] >= 6) & (test_explain['Month'] <= 8)).astype(int)

plt.figure(figsize=(12, 6))
plot_data = test_explain[(test_explain['Margin_Pct'] > -50) & (test_explain['Margin_Pct'] < 150)]

sns.scatterplot(data=plot_data, x='Mean_Volume', y='Margin_Pct', hue='Is_Summer_Peak', alpha=0.4, palette='viridis')
plt.xscale('log')
plt.axhline(0, color='red', linestyle='--', label='Zero Shortage Line')
plt.title('Adaptive LSTM: Safety Margins by Volume')
plt.xlabel('Mean Demand (Log Scale)')
plt.ylabel('Safety Margin (%)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n✅ PROCESS COMPLETE - ADAPTIVE LSTM")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject